In [65]:
#import necessary libraries

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import psycopg2
import psycopg2.extras as extras

In [15]:
#enter API credential

client_id = '8b5b459f889b464085cf9a817b80f832'
client_secret = 'c60002bede1f48708d9758527c7f1f34'

#create cursor

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [16]:

#get the track id of songs in a particular playlist

user = 'f5yfsve0ooolvin7g4i75bt9w' #this is my username
playlist_id = '37i9dQZEVXbMDoHDwVN2tF' #this is the ID of playlist: Top 50 Global

#create empty list of track IDs
ids = []
playlist = sp.user_playlist(user, playlist_id)
for item in playlist['tracks']['items']:
    track = item['track']
    ids.append(track['id'])

50


In [49]:
#get the track features of our track id list
tracks = []
for i in range(len(ids)):
  meta = sp.track(ids[i])
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  track = [name, album, artist, release_date, length, popularity]
  tracks.append(track)

# create dataframe
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity'])
df

,name,album,artist,release_date,length,popularity
0,As It Was,As It Was,Harry Styles,2022-03-31,167303,91
1,Heat Waves,Dreamland (+ Bonus Levels),Glass Animals,2020-08-06,238805,96
2,STAY (with Justin Bieber),F*CK LOVE 3: OVER YOU,The Kid LAROI,2021-07-23,141805,94
3,Envolver,Envolver,Anitta,2021-11-11,193805,98
4,Enemy (with JID) - from the series Arcane Leag...,Mercury - Act 1,Imagine Dragons,2021-09-03,173381,90
5,Cold Heart - PNAU Remix,The Lockdown Sessions,Elton John,2021-10-22,202735,79
6,Plan A,Plan A,Paulo Londra,2022-03-23,178202,93
7,abcdefu,abcdefu,GAYLE,2021-08-13,168601,100
8,INDUSTRY BABY (feat. Jack Harlow),MONTERO,Lil Nas X,2021-09-17,212352,91
9,When You're Gone,When You're Gone,Shawn Mendes,2022-03-31,172266,81


In [59]:
tracks[3][0]

'Envolver'

In [34]:
conn = psycopg2.connect("user=postgres dbname=postgres password=root")

cur = conn.cursor()

In [76]:
cur.execute("DROP TABLE IF EXISTS top50chart")

In [77]:
cur.execute("""
    CREATE TABLE top50chart(
    name text,
    album text,
    artist text,
    release_date date,
    length integer,
    popularity integer
)
""")

In [78]:
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()

In [79]:
execute_values(conn, df, 'top50chart')

the dataframe is inserted


In [86]:
sql1 = '''select * from top50chart;'''
cur.execute(sql1)
for i in cur.fetchall():
    print(i)

('As It Was', 'As It Was', 'Harry Styles', datetime.date(2022, 3, 31), 167303, 91)
('Heat Waves', 'Dreamland (+ Bonus Levels)', 'Glass Animals', datetime.date(2020, 8, 6), 238805, 96)
('STAY (with Justin Bieber)', 'F*CK LOVE 3: OVER YOU', 'The Kid LAROI', datetime.date(2021, 7, 23), 141805, 94)
('Envolver', 'Envolver', 'Anitta', datetime.date(2021, 11, 11), 193805, 98)
('Enemy (with JID) - from the series Arcane League of Legends', 'Mercury - Act 1', 'Imagine Dragons', datetime.date(2021, 9, 3), 173381, 90)
('Cold Heart - PNAU Remix', 'The Lockdown Sessions', 'Elton John', datetime.date(2021, 10, 22), 202735, 79)
('Plan A', 'Plan A', 'Paulo Londra', datetime.date(2022, 3, 23), 178202, 93)
('abcdefu', 'abcdefu', 'GAYLE', datetime.date(2021, 8, 13), 168601, 100)
('INDUSTRY BABY (feat. Jack Harlow)', 'MONTERO', 'Lil Nas X', datetime.date(2021, 9, 17), 212352, 91)
("When You're Gone", "When You're Gone", 'Shawn Mendes', datetime.date(2022, 3, 31), 172266, 81)
('MAMIII', 'MAMIII', 'Becky G'

In [83]:
def db_table_exists(conn, tablename):
    # thanks to Peter Hansen's answer for this sql
    sql = f"select * from information_schema.tables where table_name='{tablename}'" 
    
    # return results of sql query from conn as a pandas dataframe
    results_df = pd.read_sql_query(sql, conn)

    # True if we got any results back, False if we didn't
    return bool(len(results_df))

In [84]:
k = db_table_exists(conn, 'top50chart')

C:\Users\Ardian\AppData\Roaming\Python\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [85]:
k

True